# Signo UAndes

## Extracción de links

In [10]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_to_bottom(driver):
    """Desplaza la página hasta el final y espera un segundo para cargar el contenido dinámico."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Esperar un segundo para que la página cargue

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(1)

        # Obtener todos los artículos dentro de los divs con clase 'news-item'
        articles = driver.find_elements(By.CSS_SELECTOR, "div.news-item")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "a.news-item-image img")
                image_url = img.get_attribute("src")
                # Asegurarse de que la URL de la imagen sea completa
                if image_url and not image_url.startswith("http"):
                    image_url = "https://www.uandes.cl" + image_url  # Ajusta el dominio si es necesario
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "a.news-link h3.block-list-item-title")
                title = title_element.text.strip()
                link = title_element.find_element(By.XPATH, '..').get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "span.block-list-item-post-date").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()


def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}page/{current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

Noticias

In [11]:
base_url = "https://www.uandes.cl/noticias-uandes/"
noticias = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=530, headless=False)
noticias

Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/1
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/2
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/3
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/4
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/5
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/6
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/7
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/8
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/9
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/10
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/11
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/12
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/13
Extrayendo artículos de: https://www.uandes.cl/noticias-uandes/page/14
Extrayendo artí

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,33 doctores se integran al claustro académico ...,https://www.uandes.cl/noticias/33-doctores-se-...,Lunes 25 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
1,"Pelusa Orellana, reconocida como una de las 10...",https://www.uandes.cl/noticias/pelusa-orellana...,Lunes 18 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
2,UANDES entre las 4 mejores universidades de Ch...,https://www.uandes.cl/noticias/la-universidad-...,Domingo 17 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
3,Académicas de Obstetricia buscan potenciar el ...,https://www.uandes.cl/noticias/academicas-de-o...,Jueves 21 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
4,Académicos Rosa González y Orazio Descalzi son...,https://www.uandes.cl/noticias/academicos-rosa...,Jueves 14 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
...,...,...,...,...,...,...
4234,UANDES y Municipalidad de Puente Alto inaugura...,https://www.uandes.cl/noticias/universidad-de-...,Jueves 4 de Octubre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,None,2024-11-29 15:11:30
4235,Carmen Luz Valenzuela asume como primera presi...,https://www.uandes.cl/noticias/carmen-luz-vale...,Lunes 10 de Septiembre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,None,2024-11-29 15:11:31
4236,Evaluación de Adultos Mayores en Puente Alto,https://www.uandes.cl/noticias/evaluacion-de-a...,Lunes 20 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,None,2024-11-29 15:11:31
4237,Bibliotecólogas UANDES asisten a conferencia i...,https://www.uandes.cl/noticias/bibliotecologas...,Miércoles 1 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,None,2024-11-29 15:11:34


## Iteración sobre links

In [21]:
from selenium.webdriver.common.by import By

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (ajustar según la velocidad de carga)
    time.sleep(1)
    
    # Intentar obtener la información:
    
    # Título del artículo
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1.news-title").text.strip()  # Ajustado para el título.
    except:
        title = "No disponible"
    
    # Autor (extraído del <p> con la clase "post-meta")
    try:
        author = driver.find_element(By.CSS_SELECTOR, "p.post-meta").text.strip()
        author = author.replace("Escrito por ", "").strip()  # Eliminamos la parte "Escrito por"
    except:
        author = "No disponible"
    
    # Fecha de publicación (extraído del <p> con la clase "news-date")
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "p.news-date").text.strip()
    except:
        publication_date = "No disponible"
    
    # Extraer el contenido principal del artículo
    try:
        content = driver.find_element(By.CSS_SELECTOR, "div.news-content").text.strip()  # Contenido del artículo.
    except:
        content = "No disponible"
    
    # Extracto de la introducción (generalmente la primer parte del contenido)
    try:
        introduction = driver.find_element(By.CSS_SELECTOR, "div.news-content p").text.strip()  # Asumiendo el primer párrafo como introducción.
    except:
        introduction = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Autor": author,
        "Fecha de Publicación": publication_date,
        "Introducción": introduction,
        "Contenido": content,
        "Enlace": url,
    }
    
    return article_data

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for _, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            url = row['Enlace']
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


In [15]:
aver

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,33 doctores se integran al claustro académico ...,https://www.uandes.cl/noticias/33-doctores-se-...,Lunes 25 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
1,"Pelusa Orellana, reconocida como una de las 10...",https://www.uandes.cl/noticias/pelusa-orellana...,Lunes 18 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43
2,UANDES entre las 4 mejores universidades de Ch...,https://www.uandes.cl/noticias/la-universidad-...,Domingo 17 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,None,2024-11-29 14:35:43


In [20]:
aver = noticias[:3]
averinfo = process_all_links(aver)
averinfo

Extrayendo datos de artículos: 100%|██████████| 3/3 [00:15<00:00,  5.06s/it]


,Título,Autor,Fecha de Publicación,Programa,Introducción,Contenido,Enlace
0,33 doctores se integran al claustro académico ...,María Ignacia Manríquez,Lunes 25 de Noviembre de 2024,No disponible,La Universidad de los Andes celebró la incorpo...,La Universidad de los Andes celebró la incorpo...,https://www.uandes.cl/noticias/33-doctores-se-...
1,"Pelusa Orellana, reconocida como una de las 10...",Bárbara Olave,Lunes 18 de Noviembre de 2024,No disponible,La directora académica del Centro de Investiga...,La directora académica del Centro de Investiga...,https://www.uandes.cl/noticias/pelusa-orellana...
2,UANDES entre las 4 mejores universidades de Ch...,Marta Castillo,Domingo 17 de Noviembre de 2024,No disponible,La Universidad de los Andes pasa a ocupar la 4...,La Universidad de los Andes pasa a ocupar la 4...,https://www.uandes.cl/noticias/la-universidad-...


Para todos pero dara error para varias filas por fecha y programa, correr de nuevo con esa estructura de error.

In [23]:
df_info = process_all_links(noticias)
df_info

Extrayendo datos de artículos: 100%|██████████| 4239/4239 [3:25:20<00:00,  2.91s/it]  


,Título,Autor,Fecha de Publicación,Introducción,Contenido,Enlace
0,33 doctores se integran al claustro académico ...,María Ignacia Manríquez,Lunes 25 de Noviembre de 2024,La Universidad de los Andes celebró la incorpo...,La Universidad de los Andes celebró la incorpo...,https://www.uandes.cl/noticias/33-doctores-se-...
1,"Pelusa Orellana, reconocida como una de las 10...",Bárbara Olave,Lunes 18 de Noviembre de 2024,La directora académica del Centro de Investiga...,La directora académica del Centro de Investiga...,https://www.uandes.cl/noticias/pelusa-orellana...
2,UANDES entre las 4 mejores universidades de Ch...,Marta Castillo,Domingo 17 de Noviembre de 2024,La Universidad de los Andes pasa a ocupar la 4...,La Universidad de los Andes pasa a ocupar la 4...,https://www.uandes.cl/noticias/la-universidad-...
3,Académicas de Obstetricia buscan potenciar el ...,Lucas Bizarri,Jueves 21 de Noviembre de 2024,"Las profesoras Constanza Hernández, María José...","Las profesoras Constanza Hernández, María José...",https://www.uandes.cl/noticias/academicas-de-o...
4,Académicos Rosa González y Orazio Descalzi son...,Marisol Grisanti,Jueves 14 de Noviembre de 2024,"Los Paneles de Evaluación de ANID, la Agencia ...","Los Paneles de Evaluación de ANID, la Agencia ...",https://www.uandes.cl/noticias/academicos-rosa...
...,...,...,...,...,...,...
4234,UANDES y Municipalidad de Puente Alto inaugura...,Universidad de los Andes,Jueves 4 de Octubre de 2018,La Universidad de los Andes y la Municipalidad...,La Universidad de los Andes y la Municipalidad...,https://www.uandes.cl/noticias/universidad-de-...
4235,Carmen Luz Valenzuela asume como primera presi...,María Ignacia Manríquez,Lunes 10 de Septiembre de 2018,"“Estoy feliz”, asegura Carmen Luz Valenzuela P...","“Estoy feliz”, asegura Carmen Luz Valenzuela P...",https://www.uandes.cl/noticias/carmen-luz-vale...
4236,Evaluación de Adultos Mayores en Puente Alto,Universidad de los Andes,Lunes 20 de Agosto de 2018,Durante agosto un grupo interdisciplinario de ...,Durante agosto un grupo interdisciplinario de ...,https://www.uandes.cl/noticias/evaluacion-de-a...
4237,Bibliotecólogas UANDES asisten a conferencia i...,Universidad de los Andes,Miércoles 1 de Agosto de 2018,"Del 20 al 23 de agosto, Patricia Abumohor, Dir...","Del 20 al 23 de agosto, Patricia Abumohor, Dir...",https://www.uandes.cl/noticias/bibliotecologas...


In [24]:
df_concatenado = noticias.copy()

In [25]:
# Identificar las columnas que están en df_concatenado pero no en df_info
columnas_extra = [col for col in df_concatenado.columns if col not in df_info.columns]

# Seleccionar solo las columnas extra de df_concatenado
df_concatenado_extra = df_concatenado[['Enlace'] + columnas_extra]

# Hacer el merge entre df_info y df_concatenado_extra en la columna "Enlace"
df_combinado = pd.merge(df_info, df_concatenado_extra, on='Enlace', how='left')

# Ver el DataFrame combinado
df_combinado

,Título,Autor,Fecha de Publicación,Introducción,Contenido,Enlace,Fecha Publicación,URL Imagen,Fecha Consulta
0,33 doctores se integran al claustro académico ...,María Ignacia Manríquez,Lunes 25 de Noviembre de 2024,La Universidad de los Andes celebró la incorpo...,La Universidad de los Andes celebró la incorpo...,https://www.uandes.cl/noticias/33-doctores-se-...,Lunes 25 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,2024-11-29 14:35:43
1,"Pelusa Orellana, reconocida como una de las 10...",Bárbara Olave,Lunes 18 de Noviembre de 2024,La directora académica del Centro de Investiga...,La directora académica del Centro de Investiga...,https://www.uandes.cl/noticias/pelusa-orellana...,Lunes 18 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,2024-11-29 14:35:43
2,UANDES entre las 4 mejores universidades de Ch...,Marta Castillo,Domingo 17 de Noviembre de 2024,La Universidad de los Andes pasa a ocupar la 4...,La Universidad de los Andes pasa a ocupar la 4...,https://www.uandes.cl/noticias/la-universidad-...,Domingo 17 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,2024-11-29 14:35:43
3,Académicas de Obstetricia buscan potenciar el ...,Lucas Bizarri,Jueves 21 de Noviembre de 2024,"Las profesoras Constanza Hernández, María José...","Las profesoras Constanza Hernández, María José...",https://www.uandes.cl/noticias/academicas-de-o...,Jueves 21 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,2024-11-29 14:35:43
4,Académicos Rosa González y Orazio Descalzi son...,Marisol Grisanti,Jueves 14 de Noviembre de 2024,"Los Paneles de Evaluación de ANID, la Agencia ...","Los Paneles de Evaluación de ANID, la Agencia ...",https://www.uandes.cl/noticias/academicos-rosa...,Jueves 14 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,2024-11-29 14:35:43
...,...,...,...,...,...,...,...,...,...
4234,UANDES y Municipalidad de Puente Alto inaugura...,Universidad de los Andes,Jueves 4 de Octubre de 2018,La Universidad de los Andes y la Municipalidad...,La Universidad de los Andes y la Municipalidad...,https://www.uandes.cl/noticias/universidad-de-...,Jueves 4 de Octubre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,2024-11-29 15:11:30
4235,Carmen Luz Valenzuela asume como primera presi...,María Ignacia Manríquez,Lunes 10 de Septiembre de 2018,"“Estoy feliz”, asegura Carmen Luz Valenzuela P...","“Estoy feliz”, asegura Carmen Luz Valenzuela P...",https://www.uandes.cl/noticias/carmen-luz-vale...,Lunes 10 de Septiembre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,2024-11-29 15:11:31
4236,Evaluación de Adultos Mayores en Puente Alto,Universidad de los Andes,Lunes 20 de Agosto de 2018,Durante agosto un grupo interdisciplinario de ...,Durante agosto un grupo interdisciplinario de ...,https://www.uandes.cl/noticias/evaluacion-de-a...,Lunes 20 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,2024-11-29 15:11:31
4237,Bibliotecólogas UANDES asisten a conferencia i...,Universidad de los Andes,Miércoles 1 de Agosto de 2018,"Del 20 al 23 de agosto, Patricia Abumohor, Dir...","Del 20 al 23 de agosto, Patricia Abumohor, Dir...",https://www.uandes.cl/noticias/bibliotecologas...,Miércoles 1 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,2024-11-29 15:11:34


In [26]:
df_combinado.columns

Index(['Título', 'Autor', 'Fecha de Publicación', 'Introducción', 'Contenido',
       'Enlace', 'Fecha Publicación', 'URL Imagen', 'Fecha Consulta'],
      dtype='object')

In [27]:
df_combinado = df_combinado[['Título', 'Autor', 'Fecha de Publicación', 'Introducción', 'Contenido',
       'Enlace', 'Fecha Publicación', 'URL Imagen']]

In [31]:
df_combinado["Think Tank"] = "Signo UAndes"
df_combinado["Categoría"] = "Noticias"

In [32]:
df_combinado

,Título,Autor,Fecha de Publicación,Introducción,Contenido,Enlace,Fecha Publicación,URL Imagen,Think Tank,Categoría
0,33 doctores se integran al claustro académico ...,María Ignacia Manríquez,Lunes 25 de Noviembre de 2024,La Universidad de los Andes celebró la incorpo...,La Universidad de los Andes celebró la incorpo...,https://www.uandes.cl/noticias/33-doctores-se-...,Lunes 25 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
1,"Pelusa Orellana, reconocida como una de las 10...",Bárbara Olave,Lunes 18 de Noviembre de 2024,La directora académica del Centro de Investiga...,La directora académica del Centro de Investiga...,https://www.uandes.cl/noticias/pelusa-orellana...,Lunes 18 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
2,UANDES entre las 4 mejores universidades de Ch...,Marta Castillo,Domingo 17 de Noviembre de 2024,La Universidad de los Andes pasa a ocupar la 4...,La Universidad de los Andes pasa a ocupar la 4...,https://www.uandes.cl/noticias/la-universidad-...,Domingo 17 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
3,Académicas de Obstetricia buscan potenciar el ...,Lucas Bizarri,Jueves 21 de Noviembre de 2024,"Las profesoras Constanza Hernández, María José...","Las profesoras Constanza Hernández, María José...",https://www.uandes.cl/noticias/academicas-de-o...,Jueves 21 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
4,Académicos Rosa González y Orazio Descalzi son...,Marisol Grisanti,Jueves 14 de Noviembre de 2024,"Los Paneles de Evaluación de ANID, la Agencia ...","Los Paneles de Evaluación de ANID, la Agencia ...",https://www.uandes.cl/noticias/academicos-rosa...,Jueves 14 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
...,...,...,...,...,...,...,...,...,...,...
4234,UANDES y Municipalidad de Puente Alto inaugura...,Universidad de los Andes,Jueves 4 de Octubre de 2018,La Universidad de los Andes y la Municipalidad...,La Universidad de los Andes y la Municipalidad...,https://www.uandes.cl/noticias/universidad-de-...,Jueves 4 de Octubre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,Signo UAndes,Noticias
4235,Carmen Luz Valenzuela asume como primera presi...,María Ignacia Manríquez,Lunes 10 de Septiembre de 2018,"“Estoy feliz”, asegura Carmen Luz Valenzuela P...","“Estoy feliz”, asegura Carmen Luz Valenzuela P...",https://www.uandes.cl/noticias/carmen-luz-vale...,Lunes 10 de Septiembre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,Signo UAndes,Noticias
4236,Evaluación de Adultos Mayores en Puente Alto,Universidad de los Andes,Lunes 20 de Agosto de 2018,Durante agosto un grupo interdisciplinario de ...,Durante agosto un grupo interdisciplinario de ...,https://www.uandes.cl/noticias/evaluacion-de-a...,Lunes 20 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,Signo UAndes,Noticias
4237,Bibliotecólogas UANDES asisten a conferencia i...,Universidad de los Andes,Miércoles 1 de Agosto de 2018,"Del 20 al 23 de agosto, Patricia Abumohor, Dir...","Del 20 al 23 de agosto, Patricia Abumohor, Dir...",https://www.uandes.cl/noticias/bibliotecologas...,Miércoles 1 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,Signo UAndes,Noticias


In [34]:
import pandas as pd
import re

# Reemplazar caracteres ilegales en el DataFrame
def clean_illegal_characters(df):
    # Usar una expresión regular para eliminar caracteres de control
    return df.applymap(lambda x: re.sub(r'[\x00-\x1F\x7F]', '', str(x)) if isinstance(x, str) else x)

# Limpiar el DataFrame de caracteres ilegales
df_combinado2 = clean_illegal_characters(df_combinado)

In [37]:
df_combinado2

,Título,Autor,Fecha de Publicación,Introducción,Contenido,Enlace,Fecha Publicación,URL Imagen,Think Tank,Categoría
0,33 doctores se integran al claustro académico ...,María Ignacia Manríquez,Lunes 25 de Noviembre de 2024,La Universidad de los Andes celebró la incorpo...,La Universidad de los Andes celebró la incorpo...,https://www.uandes.cl/noticias/33-doctores-se-...,Lunes 25 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
1,"Pelusa Orellana, reconocida como una de las 10...",Bárbara Olave,Lunes 18 de Noviembre de 2024,La directora académica del Centro de Investiga...,La directora académica del Centro de Investiga...,https://www.uandes.cl/noticias/pelusa-orellana...,Lunes 18 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
2,UANDES entre las 4 mejores universidades de Ch...,Marta Castillo,Domingo 17 de Noviembre de 2024,La Universidad de los Andes pasa a ocupar la 4...,La Universidad de los Andes pasa a ocupar la 4...,https://www.uandes.cl/noticias/la-universidad-...,Domingo 17 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
3,Académicas de Obstetricia buscan potenciar el ...,Lucas Bizarri,Jueves 21 de Noviembre de 2024,"Las profesoras Constanza Hernández, María José...","Las profesoras Constanza Hernández, María José...",https://www.uandes.cl/noticias/academicas-de-o...,Jueves 21 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
4,Académicos Rosa González y Orazio Descalzi son...,Marisol Grisanti,Jueves 14 de Noviembre de 2024,"Los Paneles de Evaluación de ANID, la Agencia ...","Los Paneles de Evaluación de ANID, la Agencia ...",https://www.uandes.cl/noticias/academicos-rosa...,Jueves 14 de Noviembre de 2024,https://www.uandes.cl/wp-content/uploads/2024/...,Signo UAndes,Noticias
...,...,...,...,...,...,...,...,...,...,...
4234,UANDES y Municipalidad de Puente Alto inaugura...,Universidad de los Andes,Jueves 4 de Octubre de 2018,La Universidad de los Andes y la Municipalidad...,La Universidad de los Andes y la Municipalidad...,https://www.uandes.cl/noticias/universidad-de-...,Jueves 4 de Octubre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,Signo UAndes,Noticias
4235,Carmen Luz Valenzuela asume como primera presi...,María Ignacia Manríquez,Lunes 10 de Septiembre de 2018,"“Estoy feliz”, asegura Carmen Luz Valenzuela P...","“Estoy feliz”, asegura Carmen Luz Valenzuela P...",https://www.uandes.cl/noticias/carmen-luz-vale...,Lunes 10 de Septiembre de 2018,https://www.uandes.cl/wp-content/uploads/2019/...,Signo UAndes,Noticias
4236,Evaluación de Adultos Mayores en Puente Alto,Universidad de los Andes,Lunes 20 de Agosto de 2018,Durante agosto un grupo interdisciplinario de ...,Durante agosto un grupo interdisciplinario de ...,https://www.uandes.cl/noticias/evaluacion-de-a...,Lunes 20 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,Signo UAndes,Noticias
4237,Bibliotecólogas UANDES asisten a conferencia i...,Universidad de los Andes,Miércoles 1 de Agosto de 2018,"Del 20 al 23 de agosto, Patricia Abumohor, Dir...","Del 20 al 23 de agosto, Patricia Abumohor, Dir...",https://www.uandes.cl/noticias/bibliotecologas...,Miércoles 1 de Agosto de 2018,https://www.uandes.cl/wp-content/uploads/2018/...,Signo UAndes,Noticias


In [36]:
df_combinado2.to_excel("Signo UAndes.xlsx", index = False)

Segunda pasada con nueva función

## Resto de dfs

# Consolidación

# Anterior

In [13]:
ip_elm2 = ip_elm[:2]
ip_lyr2 = ip_lyr[:2]
ip_notins2 = ip_notins[:2]

In [14]:
ip_elm_contenido = process_all_links(ip_elm)
ip_elm_contenido

Extrayendo datos de: https://ideapais.cl/noticias/columna-a-donde-estan-que-no-se-ven-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/carta-al-director-el-parche-de-la-confianza-por-arnau-sarra-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-costos-de-mantener-a-toha-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-volver-a-poner-a-las-personas-en-el-centro-por-matias-domeyko-diario-el-llanquihue/
Extrayendo datos de: https://ideapais.cl/noticias/columna-solidaridad-uc-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-catolicos-y-la-conciencia-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-cae-victimas-y-opresores-por-magdalena-vergara-diario-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-es-justo-por-juan-pablo-lira-el-dinamo/
Extrayendo datos de: https://ideapais.cl/notici

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"Columna «A dónde están, que no se ven», por Cr...",IdeaPais,"Nov 21, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Dentro de las varias aristas que tiene el caso...,https://ideapais.cl/noticias/columna-a-donde-e...
1,"Carta al Director «El parche de la confianza»,...",María José Aravena,"Nov 14, 2024","María José Aravena, Noticias, Noticias Destaca...","Señor Director:\nEn su columna, Miriam Henríqu...",https://ideapais.cl/noticias/carta-al-director...
2,"Columna «Los costos de mantener a Tohá», por C...",IdeaPais,"Nov 7, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",A nadie le gusta que le digan cómo uno debe ha...,https://ideapais.cl/noticias/columna-los-costo...
3,Columna «Volver a poner a las personas en el c...,IdeaPais,"Oct 25, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace algunos días, en IdeaPaís organizamos un ...",https://ideapais.cl/noticias/columna-volver-a-...
4,"Columna «Solidaridad UC», por Cristián Stewart...",IdeaPais,"Oct 24, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace 14 años, nació un proyecto político en la...",https://ideapais.cl/noticias/columna-solidarid...
5,"Columna «Los católicos y la conciencia», por C...",IdeaPais,"Sep 26, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",El profesor Carlos Peña inició una importante ...,https://ideapais.cl/noticias/columna-los-catol...
6,"Columna «CAE: víctimas y opresores», por Magda...",IdeaPais,"Sep 18, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",La manera en que el Frente Amplio ha enarbolad...,https://ideapais.cl/noticias/columna-cae-victi...
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",IdeaPais,"Sep 17, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Hablemos de la asignación de recursos en educa...,https://ideapais.cl/noticias/columna-es-justo-...
8,"Columna «Conmemoración de hechos importantes»,...",IdeaPais,"Sep 16, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Sin duda que las primeras semanas de septiembr...,https://ideapais.cl/noticias/columna-conmemora...


In [15]:
ip_notins_contenido = process_all_links(ip_notins)
ip_notins_contenido

Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-organiza-conversatorio-sobre-desarrollo-urbano-con-alcaldes-de-puerto-montt-y-puerto-varas/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/candidatos-a-gobernador-regional-de-la-region-de-los-lagos-debatieron-en-el-evento-organizado-por-ideapais/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-invita-a-postular-al-ii-congreso-de-pensamiento-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-inaugura-sede-regional-en-los-lagos-es-la-cuarta-a-nivel-nacional/
Extrayendo datos de: https://ideapais.cl/sin-categoria/ideapais-realiza-debates-entre-candidatos-a-alcaldes-en-santa-cruz-y-marchigue/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/con-presencia-de-evelyn-matthei-el-centro-de-estudios-ideapais-realizo-el-congreso-de-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noti

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,IdeaPaís organiza conversatorio sobre desarrol...,María José Aravena,"Nov 28, 2024","María José Aravena, Noticias Destacadas Inicio...",Rodrigo Wainraihgt y Tomás Garate dialogaron j...,https://ideapais.cl/noticias-institucionales/i...
1,Candidatos a Gobernador Regional de la región ...,María José Aravena,"Oct 15, 2024","María José Aravena, Noticias Destacadas Inicio...",El debate se enfocó en temas de seguridad y co...,https://ideapais.cl/noticias-institucionales/c...
2,IdeaPaís invita a postular al II Congreso de P...,IdeaPais,"Jun 7, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",La instancia se realizará el 27 de septiembre ...,https://ideapais.cl/noticias-institucionales/i...
3,IdeaPaís inaugura sede regional en Los Lagos: ...,IdeaPais,"May 30, 2024","IdeaPais, Noticias IdeaPaís Los Lagos, Noticia...","En el evento, su director regional, Matías Dom...",https://ideapais.cl/noticias-institucionales/i...
4,IdeaPaís realiza debates entre candidatos a al...,María José Aravena,"Oct 17, 2024","María José Aravena, Noticias Destacadas Inicio...",El centro de estudios de inspiración social cr...,https://ideapais.cl/sin-categoria/ideapais-rea...
5,Con presencia de Evelyn Matthei el centro de e...,IdeaPais,"Sep 30, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",En el ex Congreso Nacional expusieron ex autor...,https://ideapais.cl/noticias-institucionales/c...
6,La versión 2024 del Encuentro de Líderes Jóven...,IdeaPais,"Jun 6, 2024","IdeaPais, Noticias Institucionales, Noticias O...","IdeaPaís, centro de estudios de inspiración so...",https://ideapais.cl/noticias-ohiggins/la-versi...
7,IdeaPaís y centros de estudios presentaron la ...,IdeaPais,"May 27, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",Con la presencia de la alcaldesa de Providenci...,https://ideapais.cl/noticias-institucionales/i...
8,Ex autoridades del Biobío entregan sus reflexi...,María José Aravena,"Oct 16, 2024","María José Aravena, Noticias Destacadas Inicio...",El panel de conversación se enfocó en los prin...,https://ideapais.cl/noticias-institucionales/e...
9,«En el Foco» de IdeaPaís analizó las eleccione...,IdeaPais,"Jul 17, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",IdeaPaís llevó a cabo el evento online «Europa...,https://ideapais.cl/noticias-institucionales/e...


Crear otra funcion

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    try:
        # Extraer el título
        title = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_inner h1").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Extraer el año y número de forma específica (de la clase correcta)
        year_issue = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_0 .et_pb_text_inner p").text.strip()
    except:
        year_issue = "No disponible"
    
    try:
        # Extraer todo el contenido del artículo, evitando que incluya el "Año y Número"
        # Asegurarse de que el selector no esté extrayendo elementos no deseados.
        paragraphs = driver.find_elements(By.CSS_SELECTOR, ".et_pb_text_2 .et_pb_text_inner p")
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Unir todos los párrafos
    except:
        content = "No disponible"
    
    try:
        # Esperar hasta que el enlace de descarga esté presente
        download_link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[text()='Descarga la revista aquí']"))
        )
        download_link = download_link_element.get_attribute("href")
    except Exception as e:
        print(f"Error al obtener el enlace de descarga: {e}")
        download_link = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Año y Número": year_issue,
        "Contenido": content,
        "Enlace de Descarga": download_link,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [17]:
ip_lyr_contenido = process_all_links(ip_lyr, headless=True)
ip_lyr_contenido

Extrayendo datos de: https://ideapais.cl/revista-raices-la-familia-un-bien-publico-vi-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-dilemas-de-la-juventud-v-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-pensar-una-centro-derecha-reformista-iv-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-reformismo-y-constitucion-iii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-desafios-constitucionales-ii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-solidaridad-i-edicion/
Extrayendo datos de: https://ideapais.cl/producto/gonzalo-vial-politica-y-crisis-social-2da-edicion-ampliada/
Error al obtener el enlace de descarga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A7556CB5+28821]
	(No symbol) [0x00007FF6A74C3840]
	(No symbol) [0x00007FF6A736578A]
	(No symbol) [0x00007FF6A73B91BE]
	(No symbol) [0x00007FF6A73B94AC]
	(No symbol) [0x00007FF6A7402647]
	(No symbol) [0x00007FF6A73DF33F]
	(No symbol) [0x0000

,Título,Año y Número,Contenido,Enlace de Descarga,Enlace
0,Revista Raíces «La familia: un bien público»,"AÑO 2024, Nº6",El motor del trabajo de IdeaPaís durante estos...,https://ideapais.cl/wp-content/uploads/2024/04...,https://ideapais.cl/revista-raices-la-familia-...
1,Revista Raíces «Dilemas de la juventud»,"AÑO 2023, Nº5",La quinta edición de Raíces está dedicada a re...,https://ideapais.cl/wp-content/uploads/2023/08...,https://ideapais.cl/revista-raices-dilemas-de-...
2,Revista Raíces «Pensar una centro derecha refo...,"AÑO 2022, Nº4","Este número de la revista Raíces, busca dar un...",https://ideapais.cl/wp-content/uploads/2023/02...,https://ideapais.cl/revista-raices-pensar-una-...
3,Revista Raíces «Reformismo y Constitución»,"AÑO 2022, Nº3",La tercera edición de la revista Raíces nos en...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-reformismo-...
4,Revista Raíces «Desafíos constitucionales»,"AÑO 2021, Nº2",La experiencia de los últimos años nos ha ense...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-desafios-co...
5,Revista Raíces «Solidaridad»,"AÑO 2020, Nº1","A diez años del nacimiento de IdeaPaís, presen...",https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-solidaridad...
6,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/gonzalo-vial-poli...
7,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/cuestiones-obreras/
8,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/constitucion-soli...
9,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/por-que-ha-fracas...


## Consolidar archivos

In [18]:
import pandas as pd

# Lista de DataFrames
dataframes = [ip_pp, ip_ml, ip_dc, ip_elm_contenido, ip_lyr_contenido, ip_notins_contenido, ip_inic]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN


In [19]:
df_final = df_concatenado.copy()

In [21]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Idea País"

df_final  = df_final[['Título', 'Enlace', 'Fecha Publicación', 'URL Imagen', 'Contenido', 'Autor', 'Fecha de Publicación', 'Categorías',
       'Año y Número', 'Enlace de Descarga']]

## Guardar archivos

In [22]:
df_final.to_excel("Idea País.xlsx", index = False)